# Implementing Simplified DES (SDES) with SAGE

In [1]:
# Note: make sure to run this cell first
import ipywidgets as widgets

## Define Expansions and Permutations

In [12]:
# The Expansions/Permutations are stored as lists of bit positions

P10_data = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6];
P8_data = [6, 3, 7, 4, 8, 5, 10, 9];
LS1_data = [2, 3, 4, 5, 1];
LS2_data = [3, 4, 5, 1, 2];
IP_data = [2, 6, 3, 1, 4, 8, 5, 7];
IPinv_data = [4, 1, 3, 5, 7, 2, 8, 6];
EP_data = [4, 1, 2, 3, 2, 3, 4, 1];
P4_data = [2, 4, 3, 1];
SW_data = [5, 6, 7, 8, 1, 2, 3, 4];

## Define S-Boxes

In [3]:
# SDES lookup tables

S0_data = [[1, 0, 3, 2],
           [3, 2, 1, 0],
           [0, 2, 1, 3],
           [3, 1, 3, 2]];

S1_data = [[0, 1, 2, 3],
           [2, 0, 1, 3],
           [3, 0, 1, 0],
           [2, 1, 0, 3]];

Permutation Helper Method

In [4]:
def ApplyPermutation(X, permutation):
    return [X[i-1] for i in permutation]

## Substitution Helper Method

In [6]:
def ApplySBox(X, SBox):
    #A row
    r = 2*X[0] + X[3];
    #A column
    c = 2*X[1] + X[2];
    #Lookup => we get a 2 bit output.
    o = SBox[r][c]; 
    return [o & 2, o & 1];

## Concatenate Two Blocks

In [7]:
def concatenate(left, right):
    ret = [left[j] for j in xrange(len(left))];
    ret.extend(right);
    return ret;

## Return Left/Right Half Bits of a Block

In [8]:
def LeftHalfBits(block):
    l = len(block);
    return [block[j] for j in xrange(l/2)];

def RightHalfBits(block):
    l = len(block);
    return [block[j] for j in xrange(l/2, l)];

## Compute XOR of Two Blocks

In [9]:
def XorBlock(block1, block2):
    l = len(block1);
    if (l != len(block2)):
        raise ValueError, "XorBlock arguments must be same length"
        #A way to implement XOR:
    return [(int(block1[j])+int(block2[j])) % 2 for j in xrange(l)];

## Key Generation

In [10]:
def SDESKeySchedule(K):
    r"""
    Expands an SDES Key (bit list) into the two round keys.
    """
    step1 = P10(K)
    step2 = LS1(LeftHalfBits(step1))
    step3 = LS1(RightHalfBits(step1))
    step4 = concatenate(step2,step3)
    k1 = P8(step4)
    step5 = LS2(step2)
    step6 = LS2(step3)
    step7 = concatenate(step5,step6)
    k2 = P8(step7)
    return k1,k2

## Compute F_K function

In [13]:
def f_K(block, K):
    r"""
    Performs the f_K function supplied block and K.
    """
    step1 = EP(RightHalfBits(block))
    step2 = XorBlock(step1,K)
    step3 = S0(LeftHalfBits(step2))
    step4 = S1(RightHalfBits(step2))
    step5 = concatenate(step3,step4)
    step6 = P4(step5)
    result = XorBlock(LeftHalfBits(block),step6)
    return result
    

## Compute Permutations

In [14]:
def P10(X):
    return ApplyPermutation(X, P10_data);

def P8(X):
    return ApplyPermutation(X, P8_data);

def IP(X):
    return ApplyPermutation(X, IP_data);

def IPinv(X):
    return ApplyPermutation(X, IPinv_data);

def EP(X):
    return ApplyPermutation(X, EP_data);

def P4(X):
    return ApplyPermutation(X, P4_data);

def SW(X): 
    return ApplyPermutation(X, SW_data);

def LS1(X):
    return ApplyPermutation(X, LS1_data);

def LS2(X):
    return ApplyPermutation(X, LS2_data);

## Compute S-Boxes

In [16]:
def S0(X):
    return ApplySBox(X, S0_data);

def S1(X):
    return ApplySBox(X, S1_data);

## Encrypt / Decrypt

In [18]:
def SDESEncrypt(plaintext_block, K):
    k1 = SDESKeySchedule(K)[0]
    k2 = SDESKeySchedule(K)[1]
    step1 = IP(plaintext_block)
    step2 = f_K(step1, k1)
    step3 = concatenate(RightHalfBits(step1),step2)
    step4 = SW(step3)
    step5 = f_K(step4, k2)
    step6 = concatenate(step5,RightHalfBits(step4))
    step7 = IPinv(step6)
    return step7
    
def SDESDecrypt(ciphertext, K):
    return SDESEncrypt(ciphertext,K)

In [19]:
#Encrypting a series of 8 bits
print SDESEncrypt("01010101","1010000010")

[0, 0, 0, '1', '0', 1, '0', '1']


In [20]:
#Decrypting a series of 8 bits
print SDESDecrypt("00010101","1010000010")

[0, 1, 0, '1', '0', 1, '0', '1']
